### LangChain으로 네이버 뉴스 기반 Q&A 애플리케이션 구축하기 - 기본편

① 질문 처리
- 질문 처리 단계에서는 사용자의 질문을 받아 이를 처리하고, 관련 데이터를 찾는 작업이 이루어집니다. 이를 위해 다음과 같은 구성 요소들이 필요합니다:
  - **데이터 소스 연결**: 질문에 대한 답변을 찾기 위해 다양한 텍스트 데이터 소스에 연결해야 합니다. LangChain은 다양한 데이터 소스와의 연결을 간편하게 설정할 수 있도록 돕습니다.
  - **데이터 인덱싱 및 검색**: 데이터 소스에서 관련 정보를 효율적으로 찾기 위해, 데이터는 인덱싱되어야 합니다. LangChain은 인덱싱 과정을 자동화하고, 사용자의 질문과 관련된 데이터를 검색하는 데 필요한 도구를 제공합니다.

② 답변 생성
- 관련 데이터를 찾은 후에는 이를 기반으로 사용자의 질문에 답변을 생성해야 합니다. 이 단계에서는 다음 구성 요소가 중요합니다:
  - 답변 생성 모델: LangChain은 고급 자연어 처리(NLP) 모델을 사용하여 검색된 데이터로부터 답변을 생성할 수 있는 기능을 제공합니다. 이러한 모델은 사용자의 질문과 검색된 데이터를 입력으로 받아, 적절한 답변을 생성합니다.

LangSmith의 역할
- LangSmith는 애플리케이션의 성능을 추적하고 분석하는 데 도움을 주는 도구입니다. Q&A 애플리케이션의 복잡성이 증가함에 따라, LangSmith는 다음과 같은 방법으로 유용하게 사용될 수 있습니다:
- 성능 모니터링: LangSmith를 사용하여 애플리케이션의 질문 처리 및 답변 생성 성능을 모니터링할 수 있습니다. 이를 통해 애플리케이션의 성능을 지속적으로 개선할 수 있습니다.
- 오류 분석: LangSmith는 애플리케이션이 잘못된 답변을 생성했을 때, 이의 원인을 분석하는 데 도움을 줍니다. 이를 통해 개발자는 애플리케이션의 오류를 정확하게 파악하고, 이를 수정하는 데 필요한 정보를 얻을 수 있습니다.

In [8]:
import os
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

# 디버깅을 위한 프로젝트명을 기입합니다.
os.environ["LANGCHAIN_PROJECT"] = "RAG TUTORIAL"

# tracing 을 위해서는 아래 코드의 주석을 해제하고 실행합니다.
# os.environ["LANGCHAIN_TRACING_V2"] = true

import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
#from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI#, OpenAIEmbeddings

- 웹 페이지의 내용을 로드하고, 텍스트를 청크로 나누어 인덱싱하는 과정을 거친 후, 관련된 텍스트 스니펫을 검색하여 새로운 내용을 생성하는 과정을 구현합니다.
- WebBaseLoader는 지정된 웹 페이지에서 필요한 부분만을 파싱하기 위해 bs4.SoupStrainer를 사용합니다.
- bs4.SoupStrainer 는 편리하게 웹에서 원하는 요소를 가져올 수 있도록 해줍니다.

In [12]:
# bs4.SoupStrainer(
#     "div",
#     attrs={"Class":["newsct_article _article_body", "media_end_head_title"]}, # 클래스 명을 입력
# )

# 뉴스기사 내용을 로드하고, 청크로 나누고, 인덱싱합니다.
loader = WebBaseLoader(
    web_paths = ("https://n.news.naver.com/article/437/0000378416", ),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class":["newsct_article _article_body",
                            "media_end_head_title"]}, # 클래스 명을 입력            
        )
    ),
)

docs = loader.load()
print(f"문서의 수: {len(docs)}")
docs # Document 클래스의 리스트 

문서의 수: 1


[Document(page_content="\n출산 직원에게 '1억원' 쏜다…회사의 파격적 저출생 정책\n\n\n[앵커]올해 아이 낳을 계획이 있는 가족이라면 솔깃할 소식입니다. 정부가 저출생 대책으로 매달 주는 부모 급여, 0세 아이는 100만원으로 올렸습니다. 여기에 첫만남이용권, 아동수당까지 더하면 아이 돌까지 1년 동안 1520만원을 받습니다. 지자체도 경쟁하듯 지원에 나섰습니다. 인천시는 새로 태어난 아기, 18살될 때까지 1억원을 주겠다. 광주시도 17살될 때까지 7400만원 주겠다고 했습니다. 선거 때면 나타나서 아이 낳으면 현금 주겠다고 밝힌 사람이 있었죠. 과거에는 표만 노린 '황당 공약'이라는 비판이 따라다녔습니다. 그런데 지금은 출산율이 이보다 더 나쁠 수 없다보니, 이런 현금성 지원을 진지하게 정책화 하는 상황까지 온 겁니다. 게다가 기업들도 뛰어들고 있습니다. 이번에는 출산한 직원에게 단번에 1억원을 주겠다는 회사까지 나타났습니다.이상화 기자가 취재했습니다.[기자]한 그룹사가 오늘 파격적인 저출생 정책을 내놨습니다.2021년 이후 태어난 직원 자녀에 1억원씩, 총 70억원을 지원하고 앞으로도 이 정책을 이어가기로 했습니다.해당 기간에 연년생과 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다.[오현석/부영그룹 직원 : 아이 키우는 데 금전적으로 많이 힘든 세상이잖아요. 교육이나 생활하는 데 큰 도움이 될 거라 생각합니다.]만약 셋째까지 낳는 경우엔 국민주택을 제공하겠다는 뜻도 밝혔습니다.[이중근/부영그룹 회장 : 3년 이내에 세 아이를 갖는 분이 나올 것이고 따라서 주택을 제공할 수 있는 계기가 될 것으로 생각하고.][조용현/부영그룹 직원 : 와이프가 셋째도 갖고 싶어 했는데 경제적 부담 때문에 부정적이었거든요. (이제) 긍정적으로 생각할 수 있을 것 같습니다.]오늘 행사에서는, 회사가 제공하는 출산장려금은 받는 직원들의 세금 부담을 고려해 정부가 면세해달라는 제안도 나왔습니다.이같은 출산장려책은 점점 확산하는 분위기입니다.법정기간보다 육아

In [15]:
# 문서를 지정된 크기의 청크로 나눕니다.
rc_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
splits = rc_splitter.split_documents(docs)
print(f"split 문서의 수: {len(splits)}")

# FAISS 혹은 Chroma와 같은 vectorstore는 이러한 청크를 바탕으로 문서의 벡터 표현을 생성합니다.
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())
# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

split 문서의 수: 3
[Document(page_content="출산 직원에게 '1억원' 쏜다…회사의 파격적 저출생 정책", metadata={'source': 'https://n.news.naver.com/article/437/0000378416'}), Document(page_content="[앵커]올해 아이 낳을 계획이 있는 가족이라면 솔깃할 소식입니다. 정부가 저출생 대책으로 매달 주는 부모 급여, 0세 아이는 100만원으로 올렸습니다. 여기에 첫만남이용권, 아동수당까지 더하면 아이 돌까지 1년 동안 1520만원을 받습니다. 지자체도 경쟁하듯 지원에 나섰습니다. 인천시는 새로 태어난 아기, 18살될 때까지 1억원을 주겠다. 광주시도 17살될 때까지 7400만원 주겠다고 했습니다. 선거 때면 나타나서 아이 낳으면 현금 주겠다고 밝힌 사람이 있었죠. 과거에는 표만 노린 '황당 공약'이라는 비판이 따라다녔습니다. 그런데 지금은 출산율이 이보다 더 나쁠 수 없다보니, 이런 현금성 지원을 진지하게 정책화 하는 상황까지 온 겁니다. 게다가 기업들도 뛰어들고 있습니다. 이번에는 출산한 직원에게 단번에 1억원을 주겠다는 회사까지 나타났습니다.이상화 기자가 취재했습니다.[기자]한 그룹사가 오늘 파격적인 저출생 정책을 내놨습니다.2021년 이후 태어난 직원 자녀에 1억원씩, 총 70억원을 지원하고 앞으로도 이 정책을 이어가기로 했습니다.해당 기간에 연년생과 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다.[오현석/부영그룹 직원 : 아이 키우는 데 금전적으로 많이 힘든 세상이잖아요. 교육이나 생활하는 데 큰 도움이 될 거라 생각합니다.]만약 셋째까지 낳는 경우엔 국민주택을 제공하겠다는 뜻도 밝혔습니다.[이중근/부영그룹 회장 : 3년 이내에 세 아이를 갖는 분이 나올 것이고 따라서 주택을 제공할 수 있는 계기가 될 것으로 생각하고.][조용현/부영그룹 직원 : 와이프가 셋째도 갖고 싶어 했는데 경제적 부담 때문에 부정적이었거든요. (이제) 긍정적으로 생각할 수 있을 것 같

In [17]:
"""
vectorstore.as_retriever()를 통해 생성된 검색기는 hub.pull로 가져온 프롬프트와
ChatOpenAI 모델을 사용하여 새로운 내용을 생성합니다.

"rlm/rag-prompt" 프롬프트는 
"""
prompt = hub.pull("rlm/rag-prompt")
print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


In [21]:
print(prompt.messages[0].prompt.template)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:


In [ ]:
from langchain.callbacks.base import BaseCallbackHandler

class StreamCallback(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs):
        print(token, end="", flush=True)

llm = ChatOpenAI(
    model_name="gpt-4-turbo-preview",
    temperature=0,
    streaming=True,
    callbacks=[StreamCallback()],
)

#llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, 
     "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# 이 함수는 rag_chain 객체의 invoke 메서드를 사용하여 사용자의 질문을 처리합니다.
# invoke??

rag_chain.invoke(
    "부영그룹의 출산 장려 정책에 대해 설명해주세요."
)  # 문서에 대한 질의를 입력하고, 답변을 출력합니다.

rag_chain.invoke(
    "부영그룹은 출산 직원에게 얼마의 지원을 제공하나요?"
)  # 문서에 대한 질의를 입력하고, 답변을 출력합니다.

rag_chain.invoke(
    "부영그룹의 임직원 숫자는 몇명인가요?"
)  # 문서에 대한 질의를 입력하고, 답변을 출력합니다.

vectorstore 객체의 delete_collection 메서드를 호출하여 컬렉션을 삭제합니다.

이는 데이터 정리 과정에서 사용되며, 특정 데이터셋이나 정보를 저장하는 컬렉션을 제거함으로써 시스템의 저장 공간을 확보하고, 불필요한 데이터로 인한 혼란을 방지합니다.

In [ ]:
vectorstore.delete_collection()

출처
- https://teddylee777.github.io/langchain/rag-naver-news-qa/

<고찰>
- 위의 튜토리얼은 기사를 읽어와 FAISS DB를 이용해 Q&A를 하는 예제이다
- 목표 프로젝트를 위해서 다음 사항에 시행 착오를 거칠 필요가 있다.
  - (1) 텍스트 Splitter 전략
    - 어떤 텍스트 Splitter를 사용할 것이냐 (보통은 RecursiveCharacterTextSplitter를 많이 사용 한다지만), chunk_size, chunk_overlab을 어떻게 할 것이냐에 따라 결과가 달라질 수 있다.
  - (2) 임베딩 모델 
    - 어떤 임베딩 모델을 사용할 것인가 (보통은 OpenAIEmbedding + CacheBackedEmbedding을 많이 사용 한다지만)에 따라 결과가 달라질 수 있다.
  - (3) 벡터 Store
    - 어떤 DB를 쓸 것인가 에 따라 결과가 상의할 수 있다
  - (4) 프롬프트
    - 프롬프트를 어떻게 설정하느냐에 따라 결과가 달라질 수 있다.
- 목표로한 프로젝트를 간단하게 만들어 보고, 카톡방에 배포 해서 반응을 들어보자